<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/Copy_of_BGL2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
#upload log files
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL100/X_train_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL100/Xabnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL100/Xnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL100/log2index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL100/reduced_index2embed'

--2023-09-07 11:40:16--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL100/X_train_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9024878 (8.6M) [text/plain]
Saving to: ‘X_train_index’

X_train_index       100%[===================>]   8.61M  --.-KB/s    in 0.07s   

2023-09-07 11:40:18 (121 MB/s) - ‘X_train_index’ saved [9024878/9024878]

--2023-09-07 11:40:18--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL100/Xabnorm_test_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respo

In [40]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [41]:
#Json file name
with open('/content/reduced_index2embed') as f:
    embeddings = [json.loads(line) for line in f.readlines()]

len(embeddings[0])

55

In [42]:
class AE(torch.nn.Module):
    def __init__(self, input_size):
        super().__init__()

        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(128, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [43]:
sequence_length = 100
input_size = len(embeddings[0]) * sequence_length


model = AE(input_size).to(device)
model

AE(
  (encoder): Sequential(
    (0): Linear(in_features=5500, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=128, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=5500, bias=True)
  )
)

In [44]:
data = torch.randn(1,input_size) #batch_size=1
print(data.shape)

model(data.to(device)).shape

torch.Size([1, 5500])


torch.Size([1, 5500])

In [46]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 12452348


In [47]:
window_size = sequence_length
num_sessions = 0
sequences = {}

line = [i for i in range(20)]
print(line)

for i in range(len(line) - window_size):
    seq = tuple(line[i:i + window_size])
    if seq not in sequences:
      sequences[seq] = [ embeddings[i] for i in seq]


print(sequences.keys())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
dict_keys([])


In [50]:
name = 'X_train_index'
window_size = sequence_length
sequences = {}
#outputs = []
with open('/content/' + name, 'r') as f:
        for row in f:
            line = [ int(i) for i in row.strip().split()]

            seq = tuple(line)
            if seq not in sequences:
              sequences[seq] = [ embeddings[i] for i in seq]

print('the length of dictionary : ', len(sequences))

dataset = TensorDataset(torch.tensor(list(sequences.values()), dtype=torch.float))

the length of dictionary :  7126


In [52]:
batch_size=128
dataloader = DataLoader(dataset, batch_size, shuffle=True)
print(len(dataloader)) # regarding to batch size


for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

56
torch.Size([128, 100, 55])


In [53]:
def adjust_learning_rate(optimizer, epoch, lr_step=(80,120,160), lr_decay_ratio=0.5):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [54]:
num_epochs = 200
learning_rate = 0.001

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [55]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in tqdm(range(num_epochs), desc="Processing Rows"):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(seq[0].shape[0], -1).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, (train_loss*100) / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Processing Rows:   0%|          | 1/200 [00:15<52:09, 15.73s/it]

Epoch [1/200], train_loss: 0.7552


Processing Rows:   1%|          | 2/200 [00:31<51:02, 15.47s/it]

Epoch [2/200], train_loss: 0.4107


Processing Rows:   2%|▏         | 3/200 [00:45<49:08, 14.97s/it]

...

Epoch [101/200], train_loss: 0.0626


Processing Rows:  51%|█████     | 102/200 [27:01<26:18, 16.10s/it]

Epoch [102/200], train_loss: 0.0624


Processing Rows:  51%|█████     | 102/200 [27:09<26:05, 15.97s/it]


KeyboardInterrupt: ignored

In [ ]:
# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/BGL1_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [ ]:
len(embeddings)

736

In [56]:
def generate(name):
    window_size = sequence_length
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [57]:
test_normal_loader, test_normal_length = generate('Xnorm_test_index')
test_abnormal_loader, test_abnormal_length = generate('Xabnorm_test_index')

Number of sessions(Xnorm_test_index): 4207
Number of sessions(Xabnorm_test_index): 1893


In [60]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in tqdm(test_normal_loader.keys(), desc="Processing Rows"):

          session = line
          seq = [embeddings[temp] for temp in session]
          seq = torch.tensor(seq, dtype=torch.float).view(1,-1).to(device)
          output = model(seq)

          loss = criterion(output, seq)

          if (loss.cpu().detach().numpy()>threshold):
            FP += test_normal_loader[line] # numbers of that set we have

  with torch.no_grad():
      for line in tqdm(test_abnormal_loader.keys(), desc="Processing Rows"):
              session = line
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(1,-1).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]

  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [61]:
threshold = [0.0006,0.0008,0.002,0.0025]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.0006


Processing Rows: 100%|██████████| 1893/1893 [00:12<00:00, 149.17it/s]


elapsed_time: 41.759s
false positive (FP): 4338, false negative (FN): 226, Precision: 51.335%, Recall: 95.294%, F1-measure: 66.725%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0008


Processing Rows: 100%|██████████| 1893/1893 [00:13<00:00, 145.38it/s]


elapsed_time: 39.727s
false positive (FP): 4205, false negative (FN): 244, Precision: 52.014%, Recall: 94.919%, F1-measure: 67.202%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.002


Processing Rows: 100%|██████████| 1893/1893 [00:12<00:00, 150.26it/s]


elapsed_time: 40.027s
false positive (FP): 3863, false negative (FN): 1203, Precision: 48.231%, Recall: 74.948%, F1-measure: 58.692%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0025


Processing Rows:  15%|█▌        | 649/4207 [00:04<00:24, 143.04it/s]


KeyboardInterrupt: ignored